In [ ]:
import os
import math
import torch
import random
import numpy as np
from pathlib import Path
from torch.cuda import empty_cache
from IPython.display import Audio
import matplotlib.pyplot as plt
from audiotoken import AudioToken, Tokenizers

from tts.long_infer import AudioSemantic, normalize_text, generate_long
from tts.infer import AudioSemantic as VanillaAudioSemantic
from common import Config as cfg
from common import ACOUSTIC, SEMANTIC, TEXT, ctx

In [ ]:
ttslib = AudioSemantic()
vanilla_ttslib = VanillaAudioSemantic()
acoustic_tokenizer = AudioToken(Tokenizers.acoustic, device='cuda:0')
semantic_tokenizer = AudioToken(Tokenizers.semantic_s, device='cuda:0')

In [ ]:
def replace_consecutive(arr):
    mask = np.concatenate(([True], arr[1:] != arr[:-1]))
    return arr[mask]

Prepare the prompts

In [ ]:
prompt_tokens = np.load('prompts/jenny_short/tokens.npz')
sa_prompt_toks_dict = {
    'source_tokens': prompt_tokens['SEMANTIC'],
    'target_tokens': prompt_tokens['ACOUSTIC']
}

ts_prompt_toks_dict = {
    'source_tokens': prompt_tokens['TEXT'],
    'target_tokens': prompt_tokens['SEMANTIC']
}

In [ ]:
sa_prompt_toks_dict['source_tokens'].shape, sa_prompt_toks_dict['target_tokens'].shape

In [ ]:
ts_prompt_toks_dict['source_tokens'].shape, ts_prompt_toks_dict['target_tokens'].shape

In [ ]:
ttslib.text_tokenizer.decode( prompt_tokens['TEXT'])

Text to semantic

In [ ]:
from datasets import load_dataset
ds = load_dataset("roneneldan/TinyStories")
print(len(ds['train']))

In [ ]:
k = random.sample(range(len(ds['train'])), 1)[0]
random_txt = ds['train'][k]['text']

print(k, random_txt)

In [ ]:
random_txt = """In the beginning the Universe was created. This has made a lot of people very angry."""

In [ ]:
sem_toks = ttslib.text_to_semantic_long(
    random_txt,
    max_source_tokens=32,
    source_overlap=16,
    temperature=0.99,
    max_new_tokens=1024,
    prompt_dict=ts_prompt_toks_dict
)

print(sem_toks.shape)

In [ ]:
auds = []

for i in range(0, sem_toks.shape[-1], 200):
    start_idx = i
    end_idx = i + 200
    try:
        print(start_idx, end_idx)
        aud = vanilla_ttslib.semantic_to_audio(sem_toks[start_idx: end_idx])
        print(aud.shape)
        auds.append(aud)
        display(Audio(aud[0], rate=24000))
    except Exception as err:
        print(err)

In [ ]:
aud = vanilla_ttslib.semantic_to_audio(sem_toks[100:])
display(Audio(aud[0], rate=24000))

Semantic to acoustic

In [ ]:
30911

In [ ]:
sem_toks.shape

In [ ]:
acoustic_tokens, st, gt = generate_long(
    model=ttslib.semantic_acoustic_model,
    source=SEMANTIC,
    target=ACOUSTIC,
    source_tokens=sem_toks,
    device='cuda:0',
    temperature=0.95,
    max_new_tokens=2048,
    max_source_tokens=256,
    source_overlap=128,
    prompt_dict=sa_prompt_toks_dict
)

In [ ]:
empty_cache()
wav = ttslib.acoustic_tokenizer.decode(torch.tensor(acoustic_tokens))
display(Audio(wav[0].cpu().numpy(), rate=24000))

In [ ]:
idx = 2

t = gt[idx] - cfg.OFFSET[ACOUSTIC]
cb1 = t[::2] < 1024
cb2 = t[1::2] >= 1024

np.where(cb1 == False), np.where(cb2 == False)

In [ ]:
x = st[-1]
print(x.shape)

temp_sem = (x[0, :] - cfg.OFFSET[SEMANTIC])
temp_aco = vanilla_ttslib.semantic_to_audio(temp_sem.cpu().numpy())
Audio(temp_aco[0], rate=24000)

Creating a prompt

! ffmpeg -y -v 0 -i LJ025-0076.wav -acodec libmp3lame -b:a 64k female_prompt_2.wav

In [ ]:
import torchaudio
import torch
from pathlib import Path
from transformers import Wav2Vec2FeatureExtractor

from audiotoken import AudioToken, Tokenizers
from tts.utils import read_audio_file, replace_consecutive

In [ ]:
def hubert_processor(audio, processor):
    return processor(
        audio,
        sampling_rate=16_000,
        return_tensors='pt'
    ).input_values[0]
processor = Wav2Vec2FeatureExtractor.from_pretrained('voidful/mhubert-base')

In [ ]:
prompt_path = 'prompts/lj_female_long/audio.wav'

In [ ]:
acoustic_tokenizer = AudioToken(Tokenizers.acoustic, device='cuda:0')
semantic_tokenizer = AudioToken(Tokenizers.semantic_s, device='cuda:0')

In [ ]:
aud = read_audio_file(Path(prompt_path), 16000)
aud = processor(aud, sampling_rate=16000)
aud = aud['input_values'][0]

In [ ]:
prompt_aco_toks = acoustic_tokenizer.encode(Path(prompt_path))
prompt_sem_toks = semantic_tokenizer.encode(aud)

prompt_sem_toks = replace_consecutive(prompt_sem_toks[0][0])

flat_aco_toks = prompt_aco_toks[0, :2, :].clone()
flat_aco_toks[1] += 1024
flat_aco_toks = torch.stack([flat_aco_toks[0], flat_aco_toks[1]], dim=1).flatten()

# txt_toks = np.array(ttslib.text_tokenizer.encode('many animals of even complex structure which live parasitically within others are wholly devoid of an alimentary cavity <period>')).astype(np.int64)
txt_toks = np.array(ttslib.text_tokenizer.encode('said meg impatiently <period>')).astype(np.int64)

In [ ]:
auds = acoustic_tokenizer.decode(prompt_aco_toks)
Audio(auds[0], rate=24000)

In [ ]:
np.savez(
    'prompts/lj_female_long/tokens.npz',
    SEMANTIC=prompt_sem_toks.numpy().astype(np.int64),
    ACOUSTIC=flat_aco_toks.numpy().astype(np.int64), 
    TEXT=txt_toks
)

In [ ]:
prev = np.load('prompts/jenny_short/tokens.npz')